In [1]:
import sys
import os
import subprocess
import shutil
# use shutil.rmtree(<path_to_folder>) to do an rm -r on the folder

import json
import csv
# import yaml
from pathlib import Path

import pandas as pd
import numpy as np

# import matplotlib as mpl
# from cycler import cycler

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set(style="darkgrid")

# This useful if I want to give unique names to directories or files
import datetime
def curr_timestamp():
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    return formatted_datetime

print('Initial imports successful')

Initial imports successful


In [3]:
# imitating example in 
# https://github.com/JJguri/bestiapop/blob/master/sample-data/ExampleMapsTasmania.ipynb

from bestiapop import bestiapop as bp

working_dir = './'

lonlat_file = working_dir + 'centroids.csv'
lonlat_df = pd.read_csv(lonlat_file)

# year_range = '1981-2010' # using this string format rather than, e.g., np.array([2010,2011,2012])
year_range = '1981-2022' # using this string format rather than, e.g., np.array([2010,2011,2012])

# these are the only climate variables supported by BestiaPop on NASAPOWER data
climate_variables = np.array(['daily_rain', 'max_temp', 'min_temp', 'radiation'])

# on NASAPOWER, it works on round numbers and xx.5 numbers only; 
# if you give it a coordinate such as 145.63 it will round to 145.5 and/or 146

action = 'generate-climate-file'
data_source = 'nasapower'
output_type = 'dataframe'

lat_range = [lonlat_df["lat"].min(), lonlat_df["lat"].max()]
lon_range = [lonlat_df["long"].min(), lonlat_df["long"].max()]

# output_path_sector = output_path + "/" + str(sector) + "/";
# Path(output_path_sector).mkdir(parents=True, exist_ok=True);

print('Fetching data for {}/{}'.format(lat_range, lon_range))
climate_data = bp.CLIMATEBEAST(
    action=action,
    data_source=data_source,
    output_path=None,
    output_type=output_type,
    input_path=None,
    climate_variables=climate_variables,
    year_range=year_range,
    lat_range=lat_range,
    lon_range=lon_range,
    multiprocessing=None
)

df = climate_data.process_records(action)



2023-05-23 19:16:56 - POPBEAST - Extracting data and converting to dataframe format


Fetching data for [38.79999542236328, 39.38333511352539]/[-122.13333892822264, -121.63333892822264]
Total Progress: 100%|##########| 2/2 [00:33<00:00, 16.86s/it]


In [4]:
df

,lon,lat,year,day,radn,maxt,mint,rain
0,-122.13,38.8,1981,1,0.00,17.81,5.72,0.24
1,-122.13,38.8,1981,2,0.00,13.12,5.53,2.01
2,-122.13,38.8,1981,3,0.00,15.65,7.00,3.38
3,-122.13,38.8,1981,4,0.00,17.73,5.69,0.07
4,-122.13,38.8,1981,5,0.00,17.63,2.34,0.18
...,...,...,...,...,...,...,...,...
61355,-121.63,39.3,2022,361,2.43,14.79,5.02,28.32
61356,-121.63,39.3,2022,362,7.69,11.87,3.95,0.08
61357,-121.63,39.3,2022,363,3.06,8.66,6.30,19.75
61358,-121.63,39.3,2022,364,1.34,18.28,8.30,16.41


In [4]:
from scipy.interpolate import interp2d

In [5]:
sectors = {}

for sector in lonlat_df.itertuples():


IndentationError: expected an indented block (1838588770.py, line 3)

In [6]:
# ["sector", "year","day","radn","maxt","mint","rain"],
sectors = []
def interpolate_and_append(rows):
  radn = interp2d(rows["lon"], rows["lat"], rows["radn"])
  maxt = interp2d(rows["lon"], rows["lat"], rows["maxt"])
  mint = interp2d(rows["lon"], rows["lat"], rows["mint"])
  rain = interp2d(rows["lon"], rows["lat"], rows["rain"])
  for row in lonlat_df.itertuples():
    sectors.append([
      row.sector,
      rows["year"].iloc[1],
      rows["day"].iloc[1],
      radn(row.long, row.lat)[0],
      maxt(row.long, row.lat)[0],
      mint(row.long, row.lat)[0],
      rain(row.long, row.lat)[0],
    ])

df.groupby(["year", "day"]).apply(interpolate_and_append)
sectors = pd.DataFrame(sectors, columns=["sector", "year","day","radn","maxt","mint","rain"])

In [8]:
import urllib.request

Path("./temperature/").mkdir(exist_ok=True)
Path("./rain/").mkdir(exist_ok=True)
for year in range(2011,2100):
    url = "https://s3.eu-west-1.amazonaws.com/data.gaezdev.aws.fao.org/res01/GFDL-ESM2M/rcp8p5/TS/tmp_GFDL-ESM2M_rcp8p5_"+str(year)+".tif"
    destination = "./temperature/"+str(year)+".tif"
    urllib.request.urlretrieve(url, destination)

    # rain data
    url = "https://s3.eu-west-1.amazonaws.com/data.gaezdev.aws.fao.org/res01/GFDL-ESM2M/rcp8p5/TS/prc_GFDL-ESM2M_rcp8p5_"+str(year)+".tif"
    destination = "./rain/"+str(year)+".tif"
    urllib.request.urlretrieve(url, destination)

KeyboardInterrupt: 

In [9]:
import rasterio

# [year, sector, total_rain, mean_temp]
future_weather = [];
for year in range(2011,2100):
    total_rain = rasterio.open("./rain/" + str(year) + ".tif", masked=True);
    mean_temp = rasterio.open("./temperature/" + str(year) + ".tif", masked=True);
    total_rain_data = total_rain.read()[0];
    mean_temp_data = mean_temp.read()[0];
    for sector in lonlat_df.itertuples():
      idx1 = total_rain.index(sector.long, sector.lat, precision=1E-6)
      idx2 = mean_temp.index(sector.long, sector.lat, precision=1E-6)
      future_weather.append([
         year,
         sector.sector,
         total_rain_data[idx1],
         mean_temp_data[idx2],
      ])

    mean_temp.close()
    total_rain.close()
future_weather = pd.DataFrame(future_weather, columns=["year", "sector", "total_rain", "mean_temp"])

In [10]:
future_weather

,year,sector,total_rain,mean_temp
0,2011,1,605,17.219999
1,2011,2,581,17.109999
2,2011,3,575,17.039999
3,2011,4,586,16.969999
4,2011,5,596,16.789999
...,...,...,...,...
3822,2099,39,491,20.129999
3823,2099,40,446,20.049999
3824,2099,41,442,19.900000
3825,2099,42,419,19.850000


In [32]:
sectors_with_data = sectors[sectors["year"] >= 2011];
# "sector", "year","day","radn","maxt","mint","rain"
generated_points = [];
for sector in range(1,44):
  selected_sector_fut = future_weather[future_weather["sector"] == sector]
  selected_sector = sectors_with_data[sectors_with_data["sector"] == sector]

  for year in range(2023,2100):
      ref_year = 2011 + (year % 11)
      selected_year = selected_sector[selected_sector["year"] == ref_year]
      mean_temp_for_gen_year = selected_sector_fut[selected_sector_fut["year"] == year]["mean_temp"].values[0]
      mean_temp_ref = selected_sector_fut[selected_sector_fut["year"] == ref_year]["mean_temp"].values[0]

      days = 366;
      if calendar.isleap(year):
        days = 367;

      for day in range(1, days):
        sample = None;
        if day > 365:
          sample = selected_year[selected_year["day"] == 365]
        else:
          sample = selected_year[selected_year["day"] == day]
        # Not the best way to do things
        # Find the % increase in the mean temp compared to the ref point
        percent_increase = (mean_temp_for_gen_year - mean_temp_ref)/mean_temp_for_gen_year;
        # Add the % to the radn, max-t, and min-t
        sample_weather = sample[["radn", "maxt", "mint"]] + (sample[["radn", "maxt", "mint"]] * percent_increase).abs()

        generated_points.append([
          sector,
          year,
          day,
          sample_weather["radn"].values[0],
          sample_weather["maxt"].values[0],
          sample_weather["mint"].values[0],
          sample["rain"].values[0] # TODO Normalize and scale to match the expected data
        ])

In [33]:
generated_points_df = pd.DataFrame(generated_points, columns=["sector", "year","day","radn","maxt","mint","unfit_rain"])
generated_points_df = generated_points_df.set_index(["sector", "year"])

In [34]:
totals = generated_points_df.groupby(["sector", "year"]).sum()

In [35]:
future_weather_i = future_weather.set_index(["sector", "year"])

future_weather_i.loc[totals.index, "total_rain"]

sector  year
1       2023    860
        2024    836
        2025    767
        2026    898
        2027    542
               ... 
43      2095    378
        2096    550
        2097    337
        2098    537
        2099    397
Name: total_rain, Length: 3311, dtype: int16

In [36]:
generated_points_df

day       radn       maxt      mint  unfit_rain
sector year                                                 
1      2023    1   7.144043  12.314356  3.711470        0.37
       2023    2   3.572021  10.630250  7.026048        0.88
       2023    3   4.698334  17.173592  6.822239        0.59
       2023    4   4.880690  16.379810  5.535024       15.63
       2023    5  10.716064  16.454897  4.751968        0.03
...          ...        ...        ...       ...         ...
43     2099  361   9.192957  17.360909  5.638586        2.38
       2099  362   9.398564  15.625959  2.362727        0.10
       2099  363   7.286205  16.173838  3.595454        0.86
       2099  364  11.505378  17.577777  2.282828        0.32
       2099  365  10.774339  16.847272  3.800909        0.94

[1209332 rows x 5 columns]

In [37]:
generated_points_df["rain"] = (generated_points_df["unfit_rain"] / totals.loc[generated_points_df.index, "unfit_rain"]) * future_weather_i.loc[generated_points_df.index, "total_rain"]

In [38]:
finalized = pd.concat([sectors, generated_points_df.reset_index()],
          ignore_index=True).sort_values(by=['year', 'day'])

In [39]:
finalized.to_csv("./results.csv")

In [42]:
for sector, data in finalized.groupby("sector"):
    if sector != 43: continue;
    f = open("1981-2099-sector-" + str(sector) + ".met", "w")
    f.write("[weather.met.weather]\n")
    sData = lonlat_df[lonlat_df["sector"] == sector]
    f.write("Latitude=")
    f.write(str(sData.lat.values[0]))
    f.write("\n")
    f.write("Longitude=")
    f.write(str(sData.long.values[0]))
    f.write("\n")
    f.write("\n")
    f.write("year day radn maxt mint rain\n")
    f.write("() () (MJ^m2) (oC) (oC) (mm)\n")

    for values in data.itertuples():
      f.write(str(values.year))
      f.write(" ")
      f.write(str(values.day))
      f.write(" ")
      f.write("{:.2f}".format(values.radn))
      f.write(" ")
      f.write("{:.2f}".format(values.maxt))
      f.write(" ")
      f.write("{:.2f}".format(values.mint))
      f.write(" ")
      f.write("{:.2f}".format(values.rain))
      f.write("\n")


# for sector in lonlat_df.itertuples():
#     values = sectors[sector.sector];
#     for i in range(0, len(values["year"])):

#     f.close()

In [28]:
import calendar

In [30]:
for year in range(2000, 2100):
    if calendar.isleap(year): print(year)

2000
2004
2008
2012
2016
2020
2024
2028
2032
2036
2040
2044
2048
2052
2056
2060
2064
2068
2072
2076
2080
2084
2088
2092
2096
